In [1]:
import time
from collections import namedtuple

import tensorflow as tf
import numpy as np

In [3]:
with open('Data/stephen_king_it.txt', 'r') as f:
    text=f.read()
vocab = sorted(set(text))
vocab_to_int = {c: i for i, c in enumerate(vocab)}
int_to_vocab = dict(enumerate(vocab))
encoded = np.array([vocab_to_int[c] for c in text], dtype=np.int32)

In [4]:
text[:110]

'Оно\nСтивен Кинг\n\n\nКороль на все времена\nВ маленьком провинциальном городке Дерри много лет назад семерым подро'

In [5]:
encoded[:110]

array([ 90, 121, 122,   1,  93, 126, 116, 110, 113, 121,   2,  86, 116,
       121, 111,   1,   1,   1,  86, 122, 124, 122, 119, 136,   2, 121,
       108,   2, 110, 125, 113,   2, 110, 124, 113, 120, 113, 121, 108,
         1,  78,   2, 120, 108, 119, 113, 121, 136, 118, 122, 120,   2,
       123, 124, 122, 110, 116, 121, 130, 116, 108, 119, 136, 121, 122,
       120,   2, 111, 122, 124, 122, 112, 118, 113,   2,  80, 113, 124,
       124, 116,   2, 120, 121, 122, 111, 122,   2, 119, 113, 126,   2,
       121, 108, 115, 108, 112,   2, 125, 113, 120, 113, 124, 135, 120,
         2, 123, 122, 112, 124, 122], dtype=int32)

In [6]:
def get_batches(arr, n_seqs, n_steps):
    '''Создаем генератор, который возвращает пакеты размером
       n_seqs x n_steps из массива arr.
       
       Аргументы
       ---------
       arr: Массив, из которого получаем пакеты
       n_seqs: Batch size, количество последовательностей в пакете
       n_steps: Sequence length, сколько "шагов" делаем в пакете
    '''
    # Считаем количество символов на пакет и количество пакетов, которое можем сформировать
    characters_per_batch = n_seqs * n_steps
    n_batches = len(arr)//characters_per_batch
    
    # Сохраняем в массиве только символы, которые позволяют сформировать целое число пакетов
    arr = arr[:n_batches * characters_per_batch]
    
    # Делаем reshape 1D -> 2D, используя n_seqs как число строк, как на картинке
    arr = arr.reshape((n_seqs, -1))
    
    for n in range(0, arr.shape[1], n_steps):
        # пакет данных, который будет подаваться на вход сети
        x = arr[:, n:n+n_steps]
        # целевой пакет, с которым будем сравнивать предсказание, получаем сдвиганием "x" на один символ вперед
        y = np.zeros_like(x)
        y[:, :-1], y[:, -1] = x[:, 1:], x[:, 0]
        yield x, y

In [7]:
batches = get_batches(encoded, 10, 50)
x, y = next(batches)
print('x\n', x[:5, :5])
print('\ny\n', y[:5, :5])

x
 [[ 90 121 122   1  93]
 [126 113 131 113 121]
 [108   2 123 122 112]
 [113 121 139   2 120]
 [119 108 112 135 110]]

y
 [[121 122   1  93 126]
 [113 131 113 121 116]
 [  2 123 122 112 127]
 [121 139   2 120 122]
 [108 112 135 110 108]]


In [8]:
def build_inputs(batch_size, num_steps):
    ''' Определяем placeholder'ы для входных, целевых данных, а также вероятности drop out
    
        Аргументы
        ---------
        batch_size: Batch size, количество последовательностей в пакете
        num_steps: Sequence length, сколько "шагов" делаем в пакете
        
    '''
    # Объявляем placeholder'ы
    inputs = tf.placeholder(tf.int32, [batch_size, num_steps], name='inputs')
    targets = tf.placeholder(tf.int32, [batch_size, num_steps], name='targets')
    
    # Placeholder для вероятности drop out
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    
    return inputs, targets, keep_prob

In [9]:
def build_lstm(lstm_size, num_layers, batch_size, keep_prob):
    ''' Строим LSTM ячейку.
    
        Аргументы
        ---------
        keep_prob: Скаляр (tf.placeholder) для dropout keep probability
        lstm_size: Размер скрытых слоев в LSTM ячейках
        num_layers: Количество LSTM слоев
        batch_size: Batch size

    '''
    ### Строим LSTM ячейку
    
    def build_cell(lstm_size, keep_prob):
        # Начинаем с базовой LSTM ячейки
        lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
        
        # Добавляем dropout к ячейке
        drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        return drop
    
    
    # Стэкируем несколько LSTM слоев для придания глубины нашему deep learning
    cell = tf.contrib.rnn.MultiRNNCell([build_cell(lstm_size, keep_prob) for _ in range(num_layers)])
    # Инициализируем начальное состояние LTSM ячейки
    initial_state = cell.zero_state(batch_size, tf.float32)
    
    return cell, initial_state

In [10]:
def build_output(lstm_output, in_size, out_size):
    ''' Строим softmax слой и возвращаем результат его работы.
    
        Аргументы
        ---------
        
        x: Входящий от LSTM тензор
        in_size: Размер входящего тензора, (кол-во LSTM юнитов скрытого слоя)
        out_size: Размер softmax слоя (объем словаря)
    
    '''

    # вытягиваем и решэйпим тензор, выполняя преобразование 3D -> 2D
    seq_output = tf.concat(lstm_output, axis=1)
    x = tf.reshape(seq_output, [-1, in_size])
    
    # Соединяем результат LTSM слоев с softmax слоем
    with tf.variable_scope('softmax'):
        softmax_w = tf.Variable(tf.truncated_normal((in_size, out_size), stddev=0.1))
        softmax_b = tf.Variable(tf.zeros(out_size))
    
    # Считаем logit-функцию
    logits = tf.matmul(x, softmax_w) + softmax_b
    # Используем функцию softmax для получения предсказания
    out = tf.nn.softmax(logits, name='predictions')
    
    return out, logits

In [11]:
def build_loss(logits, targets, lstm_size, num_classes):
    ''' Считаем функцию потери на основании значений logit-функции и целевых значений.
    
        Аргументы
        ---------
        logits: значение logit-функции
        targets: целевые значения, с которыми сравниваем предсказания
        lstm_size: Количество юнитов в LSTM слое
        num_classes: Количество классов в целевых значениях (размер словаря)
        
    '''
    # Делаем one-hot кодирование целевых значений и решейпим по образу и подобию logits
    y_one_hot = tf.one_hot(targets, num_classes)
    y_reshaped = tf.reshape(y_one_hot, logits.get_shape())
    
    # Считаем значение функции потери softmax cross entropy loss и возвращаем среднее значение
    loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_reshaped)
    loss = tf.reduce_mean(loss)
    return loss

In [12]:
def build_optimizer(loss, learning_rate, grad_clip):
    ''' Строим оптимизатор для обучения, используя обрезку градиента.
    
        Arguments:
        loss: значение функции потери
        learning_rate: параметр скорости обучения
    
    '''
    
    # Оптимизатор для обучения, обрезка градиента для контроля "взрывающихся" градиентов
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), grad_clip)
    train_op = tf.train.AdamOptimizer(learning_rate)
    optimizer = train_op.apply_gradients(zip(grads, tvars))
    
    return optimizer

In [13]:
class CharRNN:
    
    def __init__(self, num_classes, batch_size=64, num_steps=50, 
                       lstm_size=128, num_layers=2, learning_rate=0.001, 
                       grad_clip=5, sampling=False):
    
        # Мы будем использовать эту же сеть для сэмплирования (генерации текста),
        # при этом будем подавать по одному символу за один раз
        if sampling == True:
            batch_size, num_steps = 1, 1
        else:
            batch_size, num_steps = batch_size, num_steps

        tf.reset_default_graph()
        
        # Получаем input placeholder'ы
        self.inputs, self.targets, self.keep_prob = build_inputs(batch_size, num_steps)

        # Строим LSTM ячейку
        cell, self.initial_state = build_lstm(lstm_size, num_layers, batch_size, self.keep_prob)

        ### Прогоняем данные через RNN слои
        # Делаем one-hot кодирование входящих данных
        x_one_hot = tf.one_hot(self.inputs, num_classes)
        
        # Прогоняем данные через RNN и собираем результаты
        outputs, state = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=self.initial_state)
        self.final_state = state
        
        # Получаем предсказания (softmax) и результат logit-функции
        self.prediction, self.logits = build_output(outputs, lstm_size, num_classes)
        
        # Считаем потери и оптимизируем (с обрезкой градиента)
        self.loss = build_loss(self.logits, self.targets, lstm_size, num_classes)
        self.optimizer = build_optimizer(self.loss, learning_rate, grad_clip)

In [14]:
batch_size = 100        # Размер пакета
num_steps = 100         # Шагов в пакете
lstm_size = 512         # Количество LSTM юнитов в скрытом слое
num_layers = 2          # Количество LSTM слоев
learning_rate = 0.001   # Скорость обучения
keep_prob = 0.5         # Dropout keep probability

In [15]:
epochs = 20
# Сохраняться каждый N итераций
save_every_n = 200

model = CharRNN(len(vocab), batch_size=batch_size, num_steps=num_steps,
                lstm_size=lstm_size, num_layers=num_layers, 
                learning_rate=learning_rate)

saver = tf.train.Saver(max_to_keep=100)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # Можно раскомментировать строчку ниже и продолжить обучение с checkpoint'а
    #saver.restore(sess, 'checkpoints/______.ckpt')
    counter = 0
    for e in range(epochs):
        # Обучаем сеть
        new_state = sess.run(model.initial_state)
        loss = 0
        for x, y in get_batches(encoded, batch_size, num_steps):
            counter += 1
            start = time.time()
            feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: keep_prob,
                    model.initial_state: new_state}
            batch_loss, new_state, _ = sess.run([model.loss, 
                                                 model.final_state, 
                                                 model.optimizer], 
                                                 feed_dict=feed)
            
            end = time.time()
            print('Epoch: {}/{}... '.format(e+1, epochs),
                  'Training Step: {}... '.format(counter),
                  'Training loss: {:.4f}... '.format(batch_loss),
                  '{:.4f} sec/batch'.format((end-start)))
        
            if (counter % save_every_n == 0):
                saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))
    
    saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))

Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Epoch: 1/20...  Training Step: 1...  Training loss: 4.9859...  5.3194 sec/batch
Epoch: 1/20...  Training Step: 2...  Training loss: 4.9206...  5.2964 sec/batch
Epoch: 1/20...  Training Step: 3...  Training loss: 4.5920...  5.2550 sec/batch
Epoch: 1/20...  Training Step: 4...  Training loss: 5.5572...  5.2451 sec/batch
Epoch: 1/20...  Training Step: 5...  Training loss: 4.0972...  5.2643 sec/batch
Epoch: 1/20...  Training Step: 6...  Training loss: 3.9902...  5.2428 sec/batch
Epoch: 1/20...  Training Step: 7...  Training loss: 3.9576...  5.2780 sec/batch
Epoch: 1/20...  T

Epoch: 2/20...  Training Step: 93...  Training loss: 3.2862...  5.2274 sec/batch
Epoch: 2/20...  Training Step: 94...  Training loss: 3.2904...  5.2663 sec/batch
Epoch: 2/20...  Training Step: 95...  Training loss: 3.2576...  5.2471 sec/batch
Epoch: 2/20...  Training Step: 96...  Training loss: 3.2606...  5.2193 sec/batch
Epoch: 2/20...  Training Step: 97...  Training loss: 3.2792...  5.3004 sec/batch
Epoch: 2/20...  Training Step: 98...  Training loss: 3.2990...  5.2689 sec/batch
Epoch: 2/20...  Training Step: 99...  Training loss: 3.3060...  5.2990 sec/batch
Epoch: 2/20...  Training Step: 100...  Training loss: 3.2646...  5.1931 sec/batch
Epoch: 2/20...  Training Step: 101...  Training loss: 3.2713...  5.2219 sec/batch
Epoch: 2/20...  Training Step: 102...  Training loss: 3.2459...  5.2325 sec/batch
Epoch: 2/20...  Training Step: 103...  Training loss: 3.2416...  5.2395 sec/batch
Epoch: 2/20...  Training Step: 104...  Training loss: 3.2533...  5.2275 sec/batch
Epoch: 2/20...  Trainin

Epoch: 4/20...  Training Step: 193...  Training loss: 2.7712...  5.2109 sec/batch
Epoch: 4/20...  Training Step: 194...  Training loss: 2.7459...  5.1907 sec/batch
Epoch: 4/20...  Training Step: 195...  Training loss: 2.7483...  5.1977 sec/batch
Epoch: 4/20...  Training Step: 196...  Training loss: 2.7221...  5.2975 sec/batch
Epoch: 4/20...  Training Step: 197...  Training loss: 2.7181...  5.1951 sec/batch
Epoch: 4/20...  Training Step: 198...  Training loss: 2.7516...  5.1879 sec/batch
Epoch: 4/20...  Training Step: 199...  Training loss: 2.7404...  5.2050 sec/batch
Epoch: 4/20...  Training Step: 200...  Training loss: 2.7290...  5.2493 sec/batch
Epoch: 4/20...  Training Step: 201...  Training loss: 2.7255...  5.2623 sec/batch
Epoch: 4/20...  Training Step: 202...  Training loss: 2.7237...  5.1467 sec/batch
Epoch: 4/20...  Training Step: 203...  Training loss: 2.7062...  5.2262 sec/batch
Epoch: 4/20...  Training Step: 204...  Training loss: 2.7282...  5.1873 sec/batch
Epoch: 4/20...  

Epoch: 5/20...  Training Step: 293...  Training loss: 2.5763...  5.2778 sec/batch
Epoch: 5/20...  Training Step: 294...  Training loss: 2.5529...  5.1070 sec/batch
Epoch: 5/20...  Training Step: 295...  Training loss: 2.5478...  5.1821 sec/batch
Epoch: 5/20...  Training Step: 296...  Training loss: 2.5502...  5.1897 sec/batch
Epoch: 5/20...  Training Step: 297...  Training loss: 2.5548...  5.2459 sec/batch
Epoch: 5/20...  Training Step: 298...  Training loss: 2.5531...  5.1834 sec/batch
Epoch: 5/20...  Training Step: 299...  Training loss: 2.5362...  5.1762 sec/batch
Epoch: 5/20...  Training Step: 300...  Training loss: 2.4963...  5.2937 sec/batch
Epoch: 5/20...  Training Step: 301...  Training loss: 2.5254...  5.2785 sec/batch
Epoch: 5/20...  Training Step: 302...  Training loss: 2.5619...  5.1513 sec/batch
Epoch: 5/20...  Training Step: 303...  Training loss: 2.5201...  5.5982 sec/batch
Epoch: 5/20...  Training Step: 304...  Training loss: 2.5274...  5.9608 sec/batch
Epoch: 5/20...  

Epoch: 7/20...  Training Step: 393...  Training loss: 2.4013...  5.2274 sec/batch
Epoch: 7/20...  Training Step: 394...  Training loss: 2.4423...  5.1855 sec/batch
Epoch: 7/20...  Training Step: 395...  Training loss: 2.4413...  5.2581 sec/batch
Epoch: 7/20...  Training Step: 396...  Training loss: 2.4546...  5.3418 sec/batch
Epoch: 7/20...  Training Step: 397...  Training loss: 2.4402...  5.2280 sec/batch
Epoch: 7/20...  Training Step: 398...  Training loss: 2.4478...  5.2373 sec/batch
Epoch: 7/20...  Training Step: 399...  Training loss: 2.4294...  5.2519 sec/batch
Epoch: 7/20...  Training Step: 400...  Training loss: 2.3685...  5.1796 sec/batch
Epoch: 7/20...  Training Step: 401...  Training loss: 2.4194...  5.2450 sec/batch
Epoch: 7/20...  Training Step: 402...  Training loss: 2.4260...  5.2347 sec/batch
Epoch: 7/20...  Training Step: 403...  Training loss: 2.3981...  5.2825 sec/batch
Epoch: 7/20...  Training Step: 404...  Training loss: 2.3753...  5.2135 sec/batch
Epoch: 7/20...  

...

Epoch: 13/20...  Training Step: 791...  Training loss: 2.1231...  5.2307 sec/batch
Epoch: 13/20...  Training Step: 792...  Training loss: 2.1028...  5.2125 sec/batch
Epoch: 13/20...  Training Step: 793...  Training loss: 2.1062...  5.3578 sec/batch
Epoch: 13/20...  Training Step: 794...  Training loss: 2.1026...  5.2467 sec/batch
Epoch: 13/20...  Training Step: 795...  Training loss: 2.1002...  5.2557 sec/batch
Epoch: 13/20...  Training Step: 796...  Training loss: 2.0428...  5.2414 sec/batch
Epoch: 13/20...  Training Step: 797...  Training loss: 2.0847...  5.2589 sec/batch
Epoch: 13/20...  Training Step: 798...  Training loss: 2.1403...  5.2271 sec/batch
Epoch: 13/20...  Training Step: 799...  Training loss: 2.0930...  5.2366 sec/batch
Epoch: 13/20...  Training Step: 800...  Training loss: 2.0955...  5.2134 sec/batch
Epoch: 13/20...  Training Step: 801...  Training loss: 2.1079...  5.3035 sec/batch
Epoch: 13/20...  Training Step: 802...  Training loss: 2.1214...  5.1901 sec/batch
Epoc

Epoch: 15/20...  Training Step: 890...  Training loss: 2.0695...  5.2979 sec/batch
Epoch: 15/20...  Training Step: 891...  Training loss: 2.0658...  5.2501 sec/batch
Epoch: 15/20...  Training Step: 892...  Training loss: 2.0641...  5.2633 sec/batch
Epoch: 15/20...  Training Step: 893...  Training loss: 2.0746...  5.2949 sec/batch
Epoch: 15/20...  Training Step: 894...  Training loss: 2.0597...  5.2673 sec/batch
Epoch: 15/20...  Training Step: 895...  Training loss: 2.0601...  5.3867 sec/batch
Epoch: 15/20...  Training Step: 896...  Training loss: 1.9819...  5.3339 sec/batch
Epoch: 15/20...  Training Step: 897...  Training loss: 2.0401...  5.3270 sec/batch
Epoch: 15/20...  Training Step: 898...  Training loss: 2.0516...  5.3367 sec/batch
Epoch: 15/20...  Training Step: 899...  Training loss: 2.0214...  5.4240 sec/batch
Epoch: 15/20...  Training Step: 900...  Training loss: 1.9972...  5.2914 sec/batch
Epoch: 15/20...  Training Step: 901...  Training loss: 2.0274...  5.2687 sec/batch
Epoc

Epoch: 16/20...  Training Step: 989...  Training loss: 2.0252...  5.3075 sec/batch
Epoch: 16/20...  Training Step: 990...  Training loss: 1.9630...  5.2214 sec/batch
Epoch: 16/20...  Training Step: 991...  Training loss: 1.9872...  5.1923 sec/batch
Epoch: 16/20...  Training Step: 992...  Training loss: 1.9963...  5.1647 sec/batch
Epoch: 17/20...  Training Step: 993...  Training loss: 2.0387...  5.3389 sec/batch
Epoch: 17/20...  Training Step: 994...  Training loss: 1.9925...  5.2119 sec/batch
Epoch: 17/20...  Training Step: 995...  Training loss: 1.9704...  5.2649 sec/batch
Epoch: 17/20...  Training Step: 996...  Training loss: 2.0168...  5.1891 sec/batch
Epoch: 17/20...  Training Step: 997...  Training loss: 1.9616...  5.2789 sec/batch
Epoch: 17/20...  Training Step: 998...  Training loss: 1.9612...  5.2892 sec/batch
Epoch: 17/20...  Training Step: 999...  Training loss: 1.9760...  5.2650 sec/batch
Epoch: 17/20...  Training Step: 1000...  Training loss: 1.9844...  5.2691 sec/batch
Epo

Epoch: 18/20...  Training Step: 1087...  Training loss: 1.9294...  5.2843 sec/batch
Epoch: 18/20...  Training Step: 1088...  Training loss: 1.9230...  5.3644 sec/batch
Epoch: 18/20...  Training Step: 1089...  Training loss: 1.9392...  5.2405 sec/batch
Epoch: 18/20...  Training Step: 1090...  Training loss: 1.9527...  5.2579 sec/batch
Epoch: 18/20...  Training Step: 1091...  Training loss: 1.9333...  5.2429 sec/batch
Epoch: 18/20...  Training Step: 1092...  Training loss: 1.9477...  5.2838 sec/batch
Epoch: 18/20...  Training Step: 1093...  Training loss: 1.9593...  5.2988 sec/batch
Epoch: 18/20...  Training Step: 1094...  Training loss: 1.9331...  5.2351 sec/batch
Epoch: 18/20...  Training Step: 1095...  Training loss: 1.9344...  5.2698 sec/batch
Epoch: 18/20...  Training Step: 1096...  Training loss: 1.9320...  5.2442 sec/batch
Epoch: 18/20...  Training Step: 1097...  Training loss: 1.9064...  5.2391 sec/batch
Epoch: 18/20...  Training Step: 1098...  Training loss: 1.9655...  5.2965 se

Epoch: 20/20...  Training Step: 1185...  Training loss: 1.8954...  5.3111 sec/batch
Epoch: 20/20...  Training Step: 1186...  Training loss: 1.8876...  5.2566 sec/batch
Epoch: 20/20...  Training Step: 1187...  Training loss: 1.8981...  5.2563 sec/batch
Epoch: 20/20...  Training Step: 1188...  Training loss: 1.8704...  5.2619 sec/batch
Epoch: 20/20...  Training Step: 1189...  Training loss: 1.8651...  5.2545 sec/batch
Epoch: 20/20...  Training Step: 1190...  Training loss: 1.8809...  5.4369 sec/batch
Epoch: 20/20...  Training Step: 1191...  Training loss: 1.8952...  5.4208 sec/batch
Epoch: 20/20...  Training Step: 1192...  Training loss: 1.8705...  5.2775 sec/batch
Epoch: 20/20...  Training Step: 1193...  Training loss: 1.9046...  5.2266 sec/batch
Epoch: 20/20...  Training Step: 1194...  Training loss: 1.8906...  5.2812 sec/batch
Epoch: 20/20...  Training Step: 1195...  Training loss: 1.8936...  5.2567 sec/batch
Epoch: 20/20...  Training Step: 1196...  Training loss: 1.8914...  5.3268 se

In [16]:
tf.train.get_checkpoint_state('checkpoints')

model_checkpoint_path: "checkpoints/i1240_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i200_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i400_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i600_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i800_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i1000_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i1200_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i1240_l512.ckpt"

In [17]:
def pick_top_n(preds, vocab_size, top_n=5):
    p = np.squeeze(preds)
    p[np.argsort(p)[:-top_n]] = 0
    p = p / np.sum(p)
    c = np.random.choice(vocab_size, 1, p=p)[0]
    return c

In [18]:
def sample(checkpoint, n_samples, lstm_size, vocab_size, prime="Но когда дело дошло до заказов, долговязый мужчина попросил стакан минералки"):
    samples = [c for c in prime]
    model = CharRNN(len(vocab), lstm_size=lstm_size, sampling=True)
    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, checkpoint)
        new_state = sess.run(model.initial_state)
        for c in prime:
            x = np.zeros((1, 1))
            x[0,0] = vocab_to_int[c]
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

        c = pick_top_n(preds, len(vocab))
        samples.append(int_to_vocab[c])

        for i in range(n_samples):
            x[0,0] = c
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

            c = pick_top_n(preds, len(vocab))
            samples.append(int_to_vocab[c])
        
    return ''.join(samples)

In [19]:
checkpoint = 'checkpoints/i200_l512.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab))
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i200_l512.ckpt
Но когда дело дошло до заказов, долговязый мужчина попросил стакан минералки перилаль с песели пенеть нелеснелиситал, пилала ни какота, н дадаласи стотинай прелене восто весто на нименилася носнила пореране песни сос толол се посталал верасто кис стелань вере со вралала в ссоль ва тел о срерослона не варена перос врале посетолони валь о ператинь сас рета ва пелано питулано, каскол вал руди вале патосль писель потена полета посну сновилел наки палена вол нели нилань вети сномо толи, кодени прори полено, код стело тана наста сна неснили. – стито нена кану, ка колана, калестонь вале в поруле васни, сени велатось сонолась воло тонато, вако столани не ватоно с врудуле, чос товене ни срела санела, сту нались в порунино, паровесны натала, нас поде слала, птодета наль порати васти пи полилон наста слина па прови срова те сето ва не врону сто ва преднола вали, кода не вастась сато нели нала, нескототи, к прадитала нал и подусн

In [20]:
checkpoint = 'checkpoints/i600_l512.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab))
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i600_l512.ckpt
Но когда дело дошло до заказов, долговязый мужчина попросил стакан минералки выдениле, когде, как нипорожит ос прогаднитаят вогоду потолостью, поторый на него в сладало в трубы, краки отрак последал се продного него в продому, что возват с отрики в скоруд семне наченили станая. В они воздержался в пораму,, чтобе по мутил о ни в собретил сторые проводитья, что оби преглядел ни осли постома солонно столие с сельного ветерено но простел не сворал о неся друсь. Вособы встерила потельку подому что он везно всем наченико все переми не воднему на выдился. От не манел стака. Он новол,, что в обринала воруча себловела, чтобы они пособрала посто вся посмудать сладаются напрастом, и не мажелся с того стратая, что том поледел но проговать, что такой накой слево сталоная, что ни прогалился плода потами, и они на верение пери собовалась в себлаку. Нет привитил стаку предужинов, не превиния веремно не муже, что ты пличать стрить не 

In [21]:
checkpoint = 'checkpoints/i1200_l512.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab))
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i1200_l512.ckpt
Но когда дело дошло до заказов, долговязый мужчина попросил стакан минералки совсем вечерой.

Потом слушала она стоял в кортиную или не всем полосенье, и несерь он податал, что они поднялся корабрик полоскул из не травце, и все слышало о совновими. Но сонер странием собреться на крикный краски, и стел и она послучал стринными в том, что он начитал постоять в стуране, подавевае стояла, и сказал, что вернался в паскинекой как по восерда, надовала на случая в скорой ингорде полово выдать стрика. Ну сегодня в паркова, оставлял на трабинку в комнану следу.

– Так по семь посторой не возможно, но поделень плосил на потов, как от этах совительно стали, как выповнул в стороне случай откоротко соворенный в конце. – Он получал по кородени, и с кандеменного начастал он проставовал под перед старах.

Он продомнил од перекрети в перебрати в тебе провере стриных продложения не словном совремали проблиговали всерники. Потом потым, 

In [22]:
checkpoint = tf.train.latest_checkpoint('checkpoints')
samp = sample(checkpoint, 2000, lstm_size, len(vocab))
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i1240_l512.ckpt
Но когда дело дошло до заказов, долговязый мужчина попросил стакан минералки в столоне, проволила на становить, которая подалавала стакать получительно он приносил строницу на наговор в поручения, в скажених пать, как парикова в галице, посто в пересельной полестивающий, что следную стальным положивом с себе. – Подамаешь, – предворнолся на сейчас сторон но послы в там собой половали, чтобы подозровать с слова, и приходит его мне вспомнила.

Остало но после следним палаци, как слизая в простинее строго, в коночкой дела или в себе сородних испельник. Он сернила с слешкой, что он слышал по корену, ного на можел везнита, не перность,. В пустельным выдались сему выследнего в семнов другой. И не заклычну себе, и одно заполнел, на нем, не предетательно. Он успут пальчисток с кусали с намодних. Он произнес продивал, что не вернялось, испусков на нее не поспомивал на каких-то воде, к стояки педвила его с мерту, что она не мож